# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fddfd46faf0>
├── 'a' --> tensor([[-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234]])
└── 'x' --> <FastTreeValue 0x7fddfd46fa30>
    └── 'c' --> tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                        [ 1.1140, -0.3160, -0.0552,  1.4725],
                        [ 0.9332, -0.0205,  2.5310,  0.1004]])

In [4]:
t.a

tensor([[-0.4015,  1.0968, -0.2711],
        [ 1.0180,  1.2828, -0.0234]])

In [5]:
%timeit t.a

65.9 ns ± 0.0374 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fddfd46faf0>
├── 'a' --> tensor([[ 1.5211, -0.3718, -1.0555],
│                   [ 1.8466,  0.1353,  0.3143]])
└── 'x' --> <FastTreeValue 0x7fddfd46fa30>
    └── 'c' --> tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                        [ 1.1140, -0.3160, -0.0552,  1.4725],
                        [ 0.9332, -0.0205,  2.5310,  0.1004]])

In [7]:
%timeit t.a = new_value

76.3 ns ± 0.0202 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4015,  1.0968, -0.2711],
               [ 1.0180,  1.2828, -0.0234]]),
    x: Batch(
           c: tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004]]),
       ),
)

In [10]:
b.a

tensor([[-0.4015,  1.0968, -0.2711],
        [ 1.0180,  1.2828, -0.0234]])

In [11]:
%timeit b.a

56.3 ns ± 0.0604 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-9.8588e-01, -1.6748e+00, -5.1293e-01],
               [-1.4814e+00,  1.0296e-03,  1.4556e-02]]),
    x: Batch(
           c: tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.176 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 54.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 929 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 301 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdd4e0b99a0>
├── 'a' --> tensor([[[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]],
│           
│                   [[-0.4015,  1.0968, -0.2711],
│                    [ 1.0180,  1.2828, -0.0234]]])
└── 'x' --> <FastTreeValue 0x7fdd4e0b99d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdd4e0b9730>
├── 'a' --> tensor([[-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234],
│                   [-0.4015,  1.0968, -0.2711],
│                   [ 1.0180,  1.2828, -0.0234]])
└── 'x' --> <FastTreeValue 0x7fdd4e0b9af0>
    └── 'c' --> tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                        [ 1.1140, -0.3160, -0.0552,  1.4725],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 40.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 72.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2001,  2.5697,  1.7138,  1.9889],
                       [ 1.1140, -0.3160, -0.0552,  1.4725],
                       [ 0.9332, -0.0205,  2.5310,  0.1004]],
              
                      [[-0.2001,  2.5697,  1.7138,  1.9889],
                       [ 1.1140, -0.3160, -0.0552,  1.4725],
                       [ 0.9332, -0.0205,  2.5310,  0.1004]],
              
                      [[-0.2001,  2.5697,  1.7138,  1.9889],
                       [ 1.1140, -0.3160, -0.0552,  1.4725],
                       [ 0.9332, -0.0205,  2.5310,  0.1004]],
              
                      [[-0.2001,  2.5697,  1.7138,  1.9889],
                       [ 1.1140, -0.3160, -0.0552,  1.4725],
                       [ 0.9332, -0.0205,  2.5310,  0.1004]],
              
                      [[-0.2001,  2.5697,  1.7138,  1.9889],
                       [ 1.1140, -0.3160, -0.0552,  1.4725],
                       [ 0.9332, -0.0205,  2.5310,  0.1004]],

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004],
                      [-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004],
                      [-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004],
                      [-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004],
                      [-0.2001,  2.5697,  1.7138,  1.9889],
                      [ 1.1140, -0.3160, -0.0552,  1.4725],
                      [ 0.9332, -0.0205,  2.5310,  0.1004],
                      [-0.2001,  2.5697,  1.7138,  1.9889],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
